In [1]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, classification_report, ConfusionMatrixDisplay,confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

c:\Users\Willian Murayama\.conda\envs\icd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("predict_students_dropout_and_academic_success.csv",delimiter=";")
le = LabelEncoder()
df["Target"] = le.fit_transform(df["Target"])

In [3]:
# Assume df has features and df_encoded["Target"] is the encoded label
X = df.iloc[:,:-1]
y = df["Target"]

# Optional: split to test performance later
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Optuna objective function
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 450),
        "max_depth": trial.suggest_int("max_depth", 3, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    }

    clf = RandomForestClassifier(**params, random_state=42)
    # Cross-validate on training data
    start = time.time()
    score = cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy").mean()
    duration = time.time() - start
    return score, duration

# Run the Optuna study
study = optuna.create_study(directions=["maximize","minimize"])
study.optimize(objective, n_trials=50)

# Print best parameters
best_trial = sorted(study.trials, key=lambda t: (-t.values[0], t.values[1]))[0]
print("\nBest Hyperparameters:", best_trial)

# Train final model on best parameters
best_clf = RandomForestClassifier(**best_trial.params, random_state=42)
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X=X_test)
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.inverse_transform(best_clf.classes_))
disp.plot()
plt.title("Confusion Matrix")
plt.show()

# Per-class precision, recall, F1-score

print("\nClassification Report (per-class precision, recall, F1):")
# Inverse transform encoded labels back to original
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred)

# Generate report using original class labels
print(classification_report(y_test_labels, y_pred_labels, digits=4))

[I 2025-05-18 12:27:47,064] A new study created in memory with name: no-name-63f15965-ed8a-45ba-8d52-84f7d82afa50
[I 2025-05-18 12:27:51,221] Trial 0 finished with values: [0.7649070233899903, 4.151909112930298] and parameters: {'n_estimators': 137, 'max_depth': 9, 'min_samples_split': 14, 'min_samples_leaf': 2, 'max_features': 'log2'}.
[I 2025-05-18 12:28:31,999] Trial 1 finished with values: [0.7711253086567738, 40.77570152282715] and parameters: {'n_estimators': 372, 'max_depth': 10, 'min_samples_split': 13, 'min_samples_leaf': 3, 'max_features': None}.
[I 2025-05-18 12:28:40,000] Trial 2 finished with values: [0.761232309671645, 7.9992241859436035] and parameters: {'n_estimators': 339, 'max_depth': 13, 'min_samples_split': 16, 'min_samples_leaf': 17, 'max_features': 'sqrt'}.
[I 2025-05-18 12:28:58,367] Trial 3 finished with values: [0.7646257361813664, 18.36554789543152] and parameters: {'n_estimators': 253, 'max_depth': 24, 'min_samples_split': 20, 'min_samples_leaf': 20, 'max_fea


Best Hyperparameters: FrozenTrial(number=34, state=1, values=[0.7719719671725042, 8.969146013259888], datetime_start=datetime.datetime(2025, 5, 18, 12, 36, 2, 228385), datetime_complete=datetime.datetime(2025, 5, 18, 12, 36, 11, 198481), params={'n_estimators': 353, 'max_depth': 40, 'min_samples_split': 19, 'min_samples_leaf': 1, 'max_features': 'sqrt'}, user_attrs={}, system_attrs={'NSGAIISampler:generation': 0}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=450, log=False, low=10, step=1), 'max_depth': IntDistribution(high=50, log=False, low=3, step=1), 'min_samples_split': IntDistribution(high=20, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=20, log=False, low=1, step=1), 'max_features': CategoricalDistribution(choices=('sqrt', 'log2', None))}, trial_id=34, value=None)


ModuleNotFoundError: No module named 'matplotlib_inline'